<a href="https://colab.research.google.com/github/pedro-arruda09/mlp-backpropagation/blob/main/MLP/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [55]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [56]:
# Classe Network
class Network(object):
    # Função de Ativação Sigmóide
    def sigmoid(net):
        return 1.0/(1.0+np.exp(-net))

    # Função para retornar as derivadas da função Sigmóide
    def sigmoid_prime(z):
        return sigmoid(z)*(1-sigmoid(z))

    # Função de Ativação ReLU
    def relu(z):
        return np.maximum(0, z)

    # Função para retornar as derivadas da função ReLU
    def relu_prime(z):
        return (z > 0).astype(float)

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) * np.sqrt(2/x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def feedforward(self, x):
        """Retorna a saída da rede usando ReLU nas camadas ocultas e sigmoid na saída."""
        for i, (b, w) in enumerate(zip(self.biases, self.weights)):
            if i < len(self.weights) - 1:
                x = relu(np.dot(w, x) + b)  # camadas ocultas
            else:
                x = sigmoid(np.dot(w, x) + b)  # camada de saída
        return x


    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)

        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)

            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna (nabla_b, nabla_w) representando o gradiente para a função de custo."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x
        activations = [x]   # armazenar ativações camada por camada
        nets = []           # armazenar z (net input) camada por camada

        for i, (b, w) in enumerate(zip(self.biases, self.weights)):
            net = np.dot(w, activation) + b
            nets.append(net)
            if i < len(self.weights) - 1:  # camadas ocultas
                activation = relu(net)
            else:  # camada de saída
                activation = sigmoid(net)
            activations.append(activation)

        # Backward pass
        # Camada de saída -> sigmoid
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Camadas ocultas -> ReLU
        for l in range(2, self.num_layers):
            net = nets[-l]
            sp = relu_prime(net)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [57]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [58]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
# arquitecture = [784, 30, 20, 10]
# mlp = Network(arquitecture)
# mlp.SGD(training_data, 10, 32, 0.5, test_data=test_data)ta)

# Configuração 3: 64 neurônios na camada oculta
print("=== Configuração: [784, 32, 10] ===")
mlp = Network([784, 128, 64, 10])
mlp.SGD(training_data, 10, 32, 0.1, test_data=test_data)

=== Configuração: [784, 32, 10] ===
Epoch 0 : 9068 / 10000 = 90.68%
Epoch 1 : 9271 / 10000 = 92.71%
Epoch 2 : 9386 / 10000 = 93.86%
Epoch 3 : 9455 / 10000 = 94.55%
Epoch 4 : 9532 / 10000 = 95.32%
Epoch 5 : 9562 / 10000 = 95.62%
Epoch 6 : 9601 / 10000 = 96.01%
Epoch 7 : 9621 / 10000 = 96.21%
Epoch 8 : 9644 / 10000 = 96.44%
Epoch 9 : 9673 / 10000 = 96.73%


In [59]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
# arquitecture = [784, 30, 20, 10]
# mlp = Network(arquitecture)
# mlp.SGD(training_data, 10, 32, 0.5, test_data=test_data)ta)

# Configuração 3: 64 neurônios na camada oculta
print("=== Configuração: [784, 32, 10] ===")
mlp = Network([784, 128, 64, 10])
mlp.SGD(training_data, 10, 32, 0.3, test_data=test_data)

=== Configuração: [784, 32, 10] ===
Epoch 0 : 9333 / 10000 = 93.33%
Epoch 1 : 9541 / 10000 = 95.41%
Epoch 2 : 9607 / 10000 = 96.07%
Epoch 3 : 9655 / 10000 = 96.55%
Epoch 4 : 9689 / 10000 = 96.89%
Epoch 5 : 9691 / 10000 = 96.91%
Epoch 6 : 9724 / 10000 = 97.24%
Epoch 7 : 9723 / 10000 = 97.23%
Epoch 8 : 9744 / 10000 = 97.44%
Epoch 9 : 9735 / 10000 = 97.35%


In [60]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
# arquitecture = [784, 30, 20, 10]
# mlp = Network(arquitecture)
# mlp.SGD(training_data, 10, 32, 0.5, test_data=test_data)ta)

# Configuração 3: 64 neurônios na camada oculta
print("=== Configuração: [784, 16, 10] ===")
mlp = Network([784, 128, 64, 10])
mlp.SGD(training_data, 10, 32, 0.5, test_data=test_data)

=== Configuração: [784, 16, 10] ===
Epoch 0 : 9377 / 10000 = 93.77%
Epoch 1 : 9571 / 10000 = 95.71%
Epoch 2 : 9657 / 10000 = 96.57%
Epoch 3 : 9694 / 10000 = 96.94%
Epoch 4 : 9714 / 10000 = 97.14%
Epoch 5 : 9738 / 10000 = 97.38%
Epoch 6 : 9748 / 10000 = 97.48%
Epoch 7 : 9756 / 10000 = 97.56%
Epoch 8 : 9767 / 10000 = 97.67%
Epoch 9 : 9769 / 10000 = 97.69%


##**Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.1, 0.3 e 0.5
     b) Função de ativação RELU
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 784 & 16 & 10 & 784 & 32 & 10 & 784 & 64 & 10 & \\ \hline
Sigmoide  & & 91.0\%;\hspace{5mm} 92.9\%;\hspace{5mm} 93.33\%;  & & & 91.78\%;\hspace{5mm} 93.57\%;\hspace{5mm} 94.41\%; & & & 92.15\%;\hspace{5mm} 94.08\%;\hspace{5mm} 94.83\%  &  & \\ \hline
Relu  & & 92.79\%;\hspace{5mm} 93.03\%;\hspace{5mm} 94.58\%; & & & 94.35\%;\hspace{5mm} 95.35\%;\hspace{5mm} 95.64\%; & & & 95.18\%;\hspace{5mm} 96.49\%;\hspace{5mm} 96.82\%; &  & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 784 & 64\hspace{20mm}32 & 10 & 784 & 128\hspace{20mm}64 & 10 & 784 & 32\hspace{20mm}16 & 10 & \\ \hline
Sigmoide  & & 90.79\%;\hspace{5mm}94.31\%;\hspace{5mm}95.36\%; & &  & 91.1\%;\hspace{5mm}93.77\%;\hspace{5mm}95.45\%; & & & 90.7\%;\hspace{5mm}93.41\%;\hspace{5mm}94.65\%; &  & \\ \hline
Relu  & & 96.05\%;\hspace{5mm}96.1\%;\hspace{5mm}96.78\%; & &  & 96.73\%;\hspace{5mm}97.35\%;\hspace{5mm}97.69\%; & & & 94.6\%;\hspace{5mm}96.1\%;\hspace{5mm}95.67\%; &  & \\ \hline
\end{array}

2) Modifique a taxa de aprendizagem pra diminuir com o tempo. Execute com a melhor configuração encontrada. Melhorou o resultado?
